# Youtube Data API

In [1]:
!pip install google-api-python-client

   ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
   ---------------------- ----------------- 7.3/13.1 MB 37.6 MB/s eta 0:00:01
   ---------------------------------------- 13.1/13.1 MB 37.3 MB/s eta 0:00:00


In [4]:
import json
from googleapiclient.discovery import build

In [9]:
DEVELOPER_KEY = json.load(open('./keys/youtube_key.json'))['api_key']
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

youtube = build(
    YOUTUBE_API_SERVICE_NAME,
    YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY
)


In [16]:
# Get Channel object

# 1. First find Channel ID by Channel Name

request = youtube.search().list(
        part="snippet",
        maxResults=5,
        q="Barbascura eXtra",
        type="channel"
    )

response = request.execute()

channelId = response['items'][0]['id']['channelId']

channelId

'UCHi6Q3Z-5oJUC691WLlSntA'

In [17]:
# 2. Then fetch the information we need about it

request = youtube.channels().list(
    part="snippet,contentDetails",
    id= channelId
    )
response = request.execute()
uploaded_playlist = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
uploaded_playlist

'UUHi6Q3Z-5oJUC691WLlSntA'

In [31]:
channel_videos : list = list()

page_token = None

while True:
    request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            maxResults=50,
            playlistId=uploaded_playlist,
            pageToken = page_token
    )

    response = request.execute()

    channel_videos.extend(
        v['contentDetails']['videoId']
        for v in response['items']
    )

    if response.get('nextPageToken', None) is None:
        break

    page_token = response['nextPageToken']

len(channel_videos)

378

In [53]:
import time

# Get Comments for Every Video
commentators : dict[str, set[str]] = {}

for video_id in channel_videos[:10]:

    page_token = None

    video_commentators : set = set()

    while True:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            pageToken = page_token
        )

        response = request.execute()

        video_commentators.update(
            ct['snippet']['topLevelComment']['snippet']['authorDisplayName']
            for ct in response['items']
        )

        if response.get('nextPageToken', None) is None:
            break

        page_token = response['nextPageToken']

    time.sleep(0.1)
    commentators[video_id] = video_commentators


In [54]:
len(commentators), commentators

(10,
 {'oHg8ETcoXIA': {'@-GTN-',
   '@17cuzzo',
   '@19Cinta92',
   '@47luka88',
   '@AdelinaMorenos',
   '@AdrianoBoccio',
   '@AdrianoShbabbaro',
   '@AgneseLitti',
   '@Ale1000Ale1000',
   '@AlessandroCussino',
   '@AlessandroPili',
   '@Alexis.350',
   '@AliceHope78',
   '@AndreaLunaXD',
   '@AntonioEzioIgnaccolo',
   '@Arsenal_it666',
   '@Atelierlevon',
   '@BarbascuraEXtra',
   '@Belis84',
   '@BertolliRobusto',
   '@Bineta-x2b',
   '@BjornWrt',
   '@BlackRaptor11.7',
   '@BlackieJackie04',
   '@Bluedd17',
   '@BorisRomagna',
   '@COZzZMO79',
   '@Calab-96',
   '@CaladlumIlvanthar',
   '@CallMe_kiara',
   '@CarmineBentivedo',
   '@CavalierediHalo',
   '@Ceskjo',
   '@ClaudioMiaka-l5h',
   '@Cloro_Sodio',
   '@Cocchi32',
   '@CorradoGaeta-xo1wc',
   '@CostantinoNicefalo',
   '@CristianDeluca-y1g',
   '@CristianPozzessere',
   '@Danielcraft2002',
   '@DanieleRayCaporale',
   '@Dante_38',
   '@Dario_Salvi',
   '@DarkAnima270',
   '@DavideScognamiglio-ch7sr',
   '@DeadbeatDuder',
  

In [55]:
# Now use networkx to create a Network:
#   * Nodes are the Videos
#   * The Edge (a, b) is present if a and b share some commentators

In [59]:
import networkx as nx

nodes_list = list(commentators.keys())
edges_list = []

for i in range(0, len(nodes_list)):
    video1 = nodes_list[i]
    for j in range(i+1, len(nodes_list)):
        video2 = nodes_list[j]

        common_commentators = commentators[video1] & commentators[video2]

        if len(common_commentators) > 0:
            edges_list.append((video1, video2, len(common_commentators)))


edges_list


[('oHg8ETcoXIA', 'YydAr-kkobs', 42),
 ('oHg8ETcoXIA', 'xpU3TBmoZB4', 79),
 ('oHg8ETcoXIA', 'k7HHjV_IzPs', 5),
 ('oHg8ETcoXIA', '_E33gRrWUHM', 5),
 ('oHg8ETcoXIA', 'A9izmvFqfxE', 55),
 ('oHg8ETcoXIA', 'g9jHl0VOQZA', 33),
 ('oHg8ETcoXIA', 'DpsWZQIgqqU', 7),
 ('oHg8ETcoXIA', 'TJ04gk-hoTc', 9),
 ('oHg8ETcoXIA', 'SqA3egGyb2k', 45),
 ('YydAr-kkobs', 'xpU3TBmoZB4', 45),
 ('YydAr-kkobs', 'k7HHjV_IzPs', 3),
 ('YydAr-kkobs', '_E33gRrWUHM', 8),
 ('YydAr-kkobs', 'A9izmvFqfxE', 62),
 ('YydAr-kkobs', 'g9jHl0VOQZA', 24),
 ('YydAr-kkobs', 'DpsWZQIgqqU', 5),
 ('YydAr-kkobs', 'TJ04gk-hoTc', 9),
 ('YydAr-kkobs', 'SqA3egGyb2k', 41),
 ('xpU3TBmoZB4', 'k7HHjV_IzPs', 6),
 ('xpU3TBmoZB4', '_E33gRrWUHM', 4),
 ('xpU3TBmoZB4', 'A9izmvFqfxE', 84),
 ('xpU3TBmoZB4', 'g9jHl0VOQZA', 43),
 ('xpU3TBmoZB4', 'DpsWZQIgqqU', 10),
 ('xpU3TBmoZB4', 'TJ04gk-hoTc', 11),
 ('xpU3TBmoZB4', 'SqA3egGyb2k', 54),
 ('k7HHjV_IzPs', '_E33gRrWUHM', 4),
 ('k7HHjV_IzPs', 'A9izmvFqfxE', 9),
 ('k7HHjV_IzPs', 'g9jHl0VOQZA', 4),
 ('k7HHjV_IzPs